In [1]:
import os
import json
import folium
from folium.plugins import MarkerCluster

print(folium.__version__)

0.5.0


In [2]:
SOURCE_FILENAME = 'nevada-licensees-10-geocoded.json'
TARGET_FILENAME = 'nevada-licensees-15-visualized.json'
LAT = 'latitude'
LNG = 'longitude'
GEO = 'geo'
LOCATION = [38.424877,-121.5132776]

CULTIVATOR = 'cultivator'
PRODUCER = 'producer'
DISTRIBUTOR = 'distributor'
DISPENSARY = 'dispensary'
LAB = 'lab'

icon = {
    CULTIVATOR: 'green',
    PRODUCER: 'red',
    DISTRIBUTOR: 'orange',
    DISPENSARY: 'blue',
    LAB: 'purple'
}


# If the TARGET_FILENAME exists, pull in interim results
if os.path.isfile(TARGET_FILENAME):
    SOURCE_FILENAME = TARGET_FILENAME

licensees = []
print('Reading input from {}'.format(SOURCE_FILENAME))
with open(SOURCE_FILENAME, 'r') as infile:
    licensees = json.load(infile)

total_licensees = len(licensees)
print('Total licensee records: {}'.format(total_licensees))

Reading input from nevada-licensees-10-geocoded.json
Total licensee records: 253


In [3]:
print('Visualizing licensees...')
m_licensees = folium.Map(location=LOCATION, zoom_start=5)

for licensee in licensees:
    name = licensee['ME Entity Name']
    city = licensee['Loc City']
    licensee_type = licensee['type']
    popup = '{}<br/>{} in {}'.format(name, licensee_type, city)
    licensee['marker'] = folium.Marker(
        location=[licensee[LAT], licensee[LNG]],
        popup=popup,
        icon=folium.Icon(color=icon[licensee_type]))
    licensee['marker'].add_to(m_licensees)

m_licensees


Visualizing licensees...


In [4]:
print('Visualizing licensee density...')
m2 = folium.Map(location=LOCATION, zoom_start=5)

cities = {}
RADIUS_INC = 1

for licensee in licensees:
    name = licensee['ME Entity Name']
    dba  = licensee['DBA']
    city_name = licensee['Loc City']
    licensee_type = licensee['type']
    
    if city_name in cities:
        city = cities[city_name]
        city['radius'] += RADIUS_INC
        
        if licensee_type in city:
            city[licensee_type]['radius'] += RADIUS_INC
        else:
            city[licensee_type] = {'radius': RADIUS_INC}
    else:
        cities[city_name] = {
            'name': name,
            LAT: licensee[LAT],
            LNG: licensee[LNG],
            'radius': RADIUS_INC
        }

for city_name, city in cities.items():
    # Display total licensees per city
    popup = city_name + ' (' + str(city['radius']) + ')'
    folium.CircleMarker(
        location=[city[LAT], city[LNG]],
        radius=city['radius'],
        popup=popup,
        fill=True,
        fill_color='#6D4488'
    ).add_to(m2)

m2

Visualizing licensee density...


In [5]:
print('Visualizing cultivator density...')
m_cultivator = folium.Map(location=LOCATION, zoom_start=5)

# Cultivators
licensee_type = CULTIVATOR
for city_name, city in cities.items():
    if licensee_type in city:
        licensees_in_city = city[licensee_type]['radius']
        popup = licensee_type + ' (' + str(licensees_in_city) + ')'
        folium.CircleMarker(
            location=[city[LAT], city[LNG]],
            radius=licensees_in_city,
            popup=popup,
            fill=True,
            fill_color=icon[licensee_type]
        ).add_to(m_cultivator)
        
m_cultivator


Visualizing cultivator density...


In [6]:
print('Visualizing producer density...')
m_producer = folium.Map(location=LOCATION, zoom_start=5)

# Producers
licensee_type = PRODUCER
for city_name, city in cities.items():
    if licensee_type in city:
        licensees_in_city = city[licensee_type]['radius']
        popup = licensee_type + ' (' + str(licensees_in_city) + ')'
        folium.CircleMarker(
            location=[city[LAT], city[LNG]],
            radius=licensees_in_city,
            popup=popup,
            fill=True,
            fill_color=icon[licensee_type]
        ).add_to(m_producer)
        
m_producer


Visualizing producer density...


In [7]:
print('Visualizing distributor density...')
m_distributors = folium.Map(location=LOCATION, zoom_start=5)

# Distributors
licensee_type = DISTRIBUTOR
for city_name, city in cities.items():
    if licensee_type in city:
        licensees_in_city = city[licensee_type]['radius']
        popup = licensee_type + ' (' + str(licensees_in_city) + ')'
        folium.CircleMarker(
            location=[city[LAT], city[LNG]],
            radius=licensees_in_city,
            popup=popup,
            fill=True,
            fill_color=icon[licensee_type]
        ).add_to(m_distributors)
        
m_distributors

Visualizing distributor density...


In [8]:
print('Visualizing dispensary density...')
m_dispensaries = folium.Map(location=LOCATION, zoom_start=5)

# Dispensaries
licensee_type = DISPENSARY
for city_name, city in cities.items():
    if licensee_type in city:
        licensees_in_city = city[licensee_type]['radius']
        popup = licensee_type + ' (' + str(licensees_in_city) + ')'
        folium.CircleMarker(
            location=[city[LAT], city[LNG]],
            radius=licensees_in_city,
            popup=popup,
            fill=True,
            fill_color=icon[licensee_type]
        ).add_to(m_dispensaries)
        
m_dispensaries

Visualizing dispensary density...


In [9]:
print('Visualizing laboratory density...')
m_labs = folium.Map(location=LOCATION, zoom_start=5)

# Laboratories
licensee_type = LAB
for city_name, city in cities.items():
    if licensee_type in city:
        licensees_in_city = city[licensee_type]['radius']
        popup = licensee_type + ' (' + str(licensees_in_city) + ')'
        folium.CircleMarker(
            location=[city[LAT], city[LNG]],
            radius=licensees_in_city,
            popup=popup,
            fill=True,
            fill_color=icon[licensee_type]
        ).add_to(m_labs)
        
m_labs

Visualizing laboratory density...


In [10]:
print('Clustering licensee types...')
m_controls = folium.Map(location=LOCATION, zoom_start=5)
m_controls.add_tile_layer()

# Create a MarkerCluster for each licensee type
marker_clusters = {
    CULTIVATOR: MarkerCluster(),
    PRODUCER: MarkerCluster(),
    DISTRIBUTOR: MarkerCluster(),
    DISPENSARY: MarkerCluster(),
    LAB: MarkerCluster()
}

for licensee in licensees:
    marker_clusters[licensee['type']].add_child(licensee['marker'])

# Create a FeatureGroup for each licensee type
feature_groups = {
    CULTIVATOR: folium.FeatureGroup(name='Cultivators'),
    PRODUCER: folium.FeatureGroup(name='Producers'),
    DISTRIBUTOR: folium.FeatureGroup(name='Distributors'),
    DISPENSARY: folium.FeatureGroup(name='Dispensaries'),
    LAB: folium.FeatureGroup(name='Laboratories')
}

for licensee_type, feature_group in feature_groups.items():
    feature_group.add_child(marker_clusters[licensee_type])
    m_controls.add_child(feature_group)

# Add layer controls
m_controls.add_child(folium.LayerControl())
m_controls

Visualizing ...


In [11]:
m_controls.save(os.path.join('nevada-licensees-15-visual.html'))